In [1]:
import psycopg2

# --- 数据库连接配置 (与之前脚本相同) ---
DB_HOST = 'localhost'
DB_NAME = 'mydb'
DB_USER = 'alan-hopiy'
DB_PASS = ''  # 空字符串密码
DB_PORT = '5432'

def create_stock_basic_info_table():
    """
    在 PostgreSQL 数据库中创建 stock_basic_info 表。
    """
    conn = None
    cursor = None
    
    # SQL 语句创建表
    create_table_sql = """
    CREATE TABLE IF NOT EXISTS stock_basic_info (
        ts_code TEXT PRIMARY KEY,
        name TEXT NOT NULL,
        industry TEXT,         -- 示例：可选的行业信息
        market TEXT,           -- 示例：可选的市场类型
        list_date TEXT         -- 示例：可选的上市日期
    );
    """
    # 我在上面的SQL中加入了几个常用的可选字段作为示例，您可以根据需要保留或删除它们

    try:
        # 1. 建立数据库连接
        conn = psycopg2.connect(
            host=DB_HOST,
            dbname=DB_NAME,
            user=DB_USER,
            password=DB_PASS,
            port=DB_PORT
        )
        conn.autocommit = True # DDL语句（如CREATE TABLE）通常可以自动提交，或手动提交
        cursor = conn.cursor()

        # 2. 执行创建表的操作
        print(f"正在尝试在数据库 '{DB_NAME}' 中创建 'stock_basic_info' 表...")
        cursor.execute(create_table_sql)
        print("'stock_basic_info' 表已成功创建 (如果它之前不存在的话)。")

    except psycopg2.Error as e:
        print(f"数据库操作创建 'stock_basic_info' 表失败: {e}")
    except Exception as e:
        print(f"发生了一个预料之外的错误: {e}")
    finally:
        # 3. 关闭游标和连接
        if cursor:
            cursor.close()
        if conn:
            conn.close()
        print("数据库连接已关闭。")

if __name__ == '__main__':
    print("脚本将尝试创建 'stock_basic_info' 表。")
    create_stock_basic_info_table()

脚本将尝试创建 'stock_basic_info' 表。
正在尝试在数据库 'mydb' 中创建 'stock_basic_info' 表...
'stock_basic_info' 表已成功创建 (如果它之前不存在的话)。
数据库连接已关闭。


In [2]:
import tushare as ts
import psycopg2
from psycopg2.extras import execute_values # 用于批量插入

# --- Tushare Pro API Token ---
# 请确保替换为您自己的有效Token!
TUSHARE_TOKEN = 'a872d82f46046d335ccf68ef591747ff66b9a9d598b40791b80f017a'

# --- 数据库连接配置 (与之前脚本相同) ---
DB_HOST = 'localhost'
DB_NAME = 'mydb'
DB_USER = 'alan-hopiy'
DB_PASS = ''  # 空字符串密码
DB_PORT = '5432'

# --- 表和列的定义 ---
TABLE_NAME = 'stock_basic_info'
# 我们将从Tushare获取并在数据库中对应的列
# (假设您的stock_basic_info表有这些列，与之前创建脚本的示例一致)
# Tushare的字段名与我们的表列名最好一致，方便处理
# Tushare 返回的字段包括: ts_code, symbol, name, area, industry, market, list_date 等
# 我们选择插入的列：
COLUMNS_TO_INSERT = ['ts_code', 'name', 'industry', 'market', 'list_date']
# Tushare API请求的字段列表
FIELDS_FROM_TUSHARE = 'ts_code,name,industry,market,list_date' # 根据COLUMNS_TO_INSERT调整

def populate_stock_basic_info():
    """
    从Tushare获取股票基本信息并填充到 PostgreSQL 数据库中。
    """
    conn_db = None
    cursor_db = None

    try:
        # 1. 初始化Tushare Pro API
        ts.set_token(TUSHARE_TOKEN)
        pro = ts.pro_api()
        print("Tushare Pro API 初始化成功。")

        # 2. 从Tushare获取股票基本数据
        # list_status='L' 表示上市状态的股票
        print(f"正在从Tushare获取股票基本信息 (字段: {FIELDS_FROM_TUSHARE})...")
        stock_data_df = pro.stock_basic(list_status='L', fields=FIELDS_FROM_TUSHARE)
        
        if stock_data_df is None or stock_data_df.empty:
            print("从Tushare获取股票基本信息失败或结果为空。请检查Token和网络。")
            return

        print(f"成功从Tushare获取了 {len(stock_data_df)} 条股票基本信息。")
        # print("数据预览：")
        # print(stock_data_df.head())

        # 3. 准备要插入数据库的数据
        # 确保DataFrame中的列顺序与COLUMNS_TO_INSERT一致，如果需要的话
        # stock_data_df = stock_data_df[COLUMNS_TO_INSERT] # 如果列名和顺序可能不匹配则需要这行

        # 将DataFrame转换为元组列表以进行批量插入
        # 处理NaN/NaT值，数据库中对应列如果允许NULL则可以，否则需要处理
        # 例如，对于TEXT列，可以将NaN替换为空字符串或None
        # psycopg2 在遇到 pandas 的 NaT (Not a Time) 时可能会出问题，但 list_date 是文本，应该还好
        # Tushare 返回的空值可能是 None 或空字符串，psycopg2 通常能正确处理为 NULL
        data_to_insert = [tuple(row) for row in stock_data_df[COLUMNS_TO_INSERT].values]
        
        if not data_to_insert:
            print("没有数据需要插入。")
            return

        # 4. 连接数据库并插入数据
        conn_db = psycopg2.connect(
            host=DB_HOST,
            dbname=DB_NAME,
            user=DB_USER,
            password=DB_PASS,
            port=DB_PORT
        )
        cursor_db = conn_db.cursor()
        print(f"已连接到数据库 '{DB_NAME}'。")

        # 构建批量插入的SQL语句
        # ON CONFLICT (ts_code) DO UPDATE 用于处理已存在的股票，更新其信息
        # EXCLUDED 代表尝试插入但发生冲突的值
        cols_str = ", ".join(COLUMNS_TO_INSERT)
        update_set_str = ", ".join([f"{col} = EXCLUDED.{col}" for col in COLUMNS_TO_INSERT if col != 'ts_code'])
        
        # 如果COLUMNS_TO_INSERT只包含ts_code，update_set_str会为空，需要处理
        if not update_set_str: # 理论上不会，因为name等字段会不同
            upsert_sql = f"""
                INSERT INTO {TABLE_NAME} ({cols_str})
                VALUES %s
                ON CONFLICT (ts_code) DO NOTHING; 
            """
        else:
            upsert_sql = f"""
                INSERT INTO {TABLE_NAME} ({cols_str})
                VALUES %s
                ON CONFLICT (ts_code) DO UPDATE SET {update_set_str};
            """
        
        print("正在将数据批量插入/更新到数据库...")
        # print(f"执行SQL: {upsert_sql[:200]} ... VALUES %s ...") # 打印部分SQL用于调试

        execute_values(cursor_db, upsert_sql, data_to_insert, page_size=1000) # page_size可以调整
        conn_db.commit()
        print(f"成功将 {len(data_to_insert)} 条记录插入/更新到 '{TABLE_NAME}' 表。")

    except ts.exceptions.RequestException as e_ts:
        print(f"Tushare API 请求错误: {e_ts}")
    except psycopg2.Error as e_db:
        if conn_db:
            conn_db.rollback()
        print(f"数据库操作错误: {e_db}")
        # print(f"PGCode: {e_db.pgcode}, PGError: {e_db.pgerror}")
    except Exception as e:
        if conn_db:
            conn_db.rollback()
        print(f"发生了一个未知错误: {e}")
    finally:
        if cursor_db:
            cursor_db.close()
        if conn_db:
            conn_db.close()
        print("数据库连接已关闭。")

if __name__ == '__main__':
    print("开始执行填充 'stock_basic_info' 表的脚本...")
    print(f"将从Tushare获取数据并存入表 '{TABLE_NAME}' 的列: {', '.join(COLUMNS_TO_INSERT)}")
    print(f"数据库目标: '{DB_NAME}' on '{DB_HOST}'")
    print("重要提示：请确保已将脚本中的 'YOUR_TUSHARE_TOKEN' 替换为您的真实有效Token。")
    
    # 实际运行时取消下面这行的注释
    populate_stock_basic_info()
    
    # 为了安全，默认不直接运行，提示用户取消注释
    print("-" * 50)
    print("请在使用前仔细检查Tushare Token和代码逻辑。")
    print("要实际运行数据填充，请取消 `if __name__ == '__main__':` 部分中对 `populate_stock_basic_info()` 调用的注释。")

开始执行填充 'stock_basic_info' 表的脚本...
将从Tushare获取数据并存入表 'stock_basic_info' 的列: ts_code, name, industry, market, list_date
数据库目标: 'mydb' on 'localhost'
重要提示：请确保已将脚本中的 'YOUR_TUSHARE_TOKEN' 替换为您的真实有效Token。
Tushare Pro API 初始化成功。
正在从Tushare获取股票基本信息 (字段: ts_code,name,industry,market,list_date)...
成功从Tushare获取了 5413 条股票基本信息。
已连接到数据库 'mydb'。
正在将数据批量插入/更新到数据库...
成功将 5413 条记录插入/更新到 'stock_basic_info' 表。
数据库连接已关闭。
--------------------------------------------------
请在使用前仔细检查Tushare Token和代码逻辑。
要实际运行数据填充，请取消 `if __name__ == '__main__':` 部分中对 `populate_stock_basic_info()` 调用的注释。


In [3]:
import psycopg2

# --- 数据库连接配置 (与之前脚本相同) ---
DB_HOST = 'localhost'
DB_NAME = 'mydb'
DB_USER = 'alan-hopiy'
DB_PASS = ''  # 空字符串密码
DB_PORT = '5432'

TABLE_NAME = 'stock_basic_info'

def verify_stock_basic_data():
    """
    连接到 PostgreSQL 数据库并检验 stock_basic_info 表中的数据。
    """
    conn = None
    cursor = None

    try:
        # 1. 建立数据库连接
        conn = psycopg2.connect(
            host=DB_HOST,
            dbname=DB_NAME,
            user=DB_USER,
            password=DB_PASS,
            port=DB_PORT
        )
        cursor = conn.cursor()
        print(f"已连接到数据库 '{DB_NAME}'。")

        # 2. 查询表中的总记录数
        count_sql = f"SELECT COUNT(*) FROM {TABLE_NAME};"
        cursor.execute(count_sql)
        total_rows = cursor.fetchone()[0]
        print(f"表 '{TABLE_NAME}' 中的总记录数: {total_rows}")

        if total_rows > 0:
            # 3. 查询并显示表中的前5条记录作为样本
            # 我们假设表包含这些列，与之前创建和填充时一致
            sample_sql = f"SELECT ts_code, name, industry, market, list_date FROM {TABLE_NAME} LIMIT 5;"
            print(f"\n正在从 '{TABLE_NAME}' 表中获取前5条样本数据...")
            cursor.execute(sample_sql)
            sample_rows = cursor.fetchall()
            
            if sample_rows:
                print("样本数据:")
                # 打印表头 (根据我们预期的列)
                header = ["ts_code", "name", "industry", "market", "list_date"]
                print(f"{header[0]:<12} | {header[1]:<20} | {header[2]:<15} | {header[3]:<10} | {header[4]:<10}")
                print("-" * (12 + 3 + 20 + 3 + 15 + 3 + 10 + 3 + 10))
                for row in sample_rows:
                    # 格式化输出，假设都是字符串或可以转为字符串
                    # 注意处理None值，避免打印 "None" 字符串，可以选择打印空字符串或特定标记
                    formatted_row = [str(item) if item is not None else '' for item in row]
                    print(f"{formatted_row[0]:<12} | {formatted_row[1]:<20} | {formatted_row[2]:<15} | {formatted_row[3]:<10} | {formatted_row[4]:<10}")
            else:
                print("表中没有数据可供显示样本。")
        else:
            print(f"表 '{TABLE_NAME}' 中没有数据。这与之前的填充日志不符，请检查。")


    except psycopg2.Error as e_db:
        print(f"数据库操作错误: {e_db}")
    except Exception as e:
        print(f"发生了一个未知错误: {e}")
    finally:
        if cursor:
            cursor.close()
        if conn:
            conn.close()
        print("\n数据库连接已关闭。")

if __name__ == '__main__':
    print(f"开始检验数据库表 '{TABLE_NAME}' 的数据...")
    verify_stock_basic_data()

开始检验数据库表 'stock_basic_info' 的数据...
已连接到数据库 'mydb'。
表 'stock_basic_info' 中的总记录数: 5413

正在从 'stock_basic_info' 表中获取前5条样本数据...
样本数据:
ts_code      | name                 | industry        | market     | list_date 
-------------------------------------------------------------------------------
000001.SZ    | 平安银行                 | 银行              | 主板         | 19910403  
000002.SZ    | 万科A                  | 全国地产            | 主板         | 19910129  
000004.SZ    | *ST国华                | 软件服务            | 主板         | 19910114  
000006.SZ    | 深振业A                 | 区域地产            | 主板         | 19920427  
000007.SZ    | 全新好                  | 其他商业            | 主板         | 19920413  

数据库连接已关闭。
